In [2]:
"""
Generate an 8-bit mono WAV from a NeuroKit2 ECG for Arduino/SimulIDE.
Also includes an optional exporter to a C header array (uint8_t).
pip install neurokit2 numpy
"""

import numpy as np
import wave
from pathlib import Path

# ---------------------------
# WAV writer: 8-bit unsigned
# ---------------------------
def write_wav_uint8_mono(filename: str, data_uint8: np.ndarray, fs: int) -> Path:
    data_uint8 = np.asarray(data_uint8, dtype=np.uint8).ravel()
    with wave.open(filename, 'wb') as wf:
        wf.setparams((1, 1, int(fs), data_uint8.size, "NONE", "not compressed"))
        wf.writeframes(data_uint8.tobytes())
    return Path(filename).resolve()

# --------------------------------------
# Scaling: float -> uint8 (0..255)
# --------------------------------------
def to_uint8_pcm(x: np.ndarray, peak: float = 0.95) -> np.ndarray:
    x = np.asarray(x, dtype=float).ravel()
    x = x - np.mean(x)                               # remove DC
    maxabs = np.max(np.abs(x)) or 1.0
    x = np.clip((x / maxabs) * peak, -1.0, 1.0)      # normalize with margin
    y = ((x + 1.0) * 127.5).astype(np.uint8)         # map [-1,1] -> [0,255]
    return y

# --------------------------------------
# NeuroKit2 ECG -> 8-bit WAV (robust)
# --------------------------------------
def neurokit_ecg_to_wav8(
    filename: str = "ecg_10s_8bit_500Hz.wav",
    fs: int = 500,
    duration_s: float = 10.0,
    method: str = "ecgsyn",
    heart_rate: float = 70.0,
    peak: float = 0.95
) -> Path:
    import neurokit2 as nk

    # Force integer length to avoid TypeError in some NK2 versions
    length = int(round(float(duration_s) * int(fs)))
    ecg = nk.ecg_simulate(length=length, sampling_rate=int(fs),
                          method=method, heart_rate=heart_rate)

    y_uint8 = to_uint8_pcm(ecg, peak=peak)
    return write_wav_uint8_mono(filename, y_uint8, fs)

# --------------------------------------
# Any array -> 8-bit WAV
# --------------------------------------
def array_ecg_to_wav8(ecg_array: np.ndarray, filename: str, fs: int, peak: float = 0.95) -> Path:
    y_uint8 = to_uint8_pcm(ecg_array, peak=peak)
    return write_wav_uint8_mono(filename, y_uint8, fs)

# --------------------------------------
# Optional: export C header for Arduino
# --------------------------------------
def export_uint8_as_c_header(data_uint8: np.ndarray, fs: int, header_path: str, var_name: str = "ecg_data") -> Path:
    data_uint8 = np.asarray(data_uint8, dtype=np.uint8).ravel()
    lines = []
    lines.append("// Auto-generated ECG samples (8-bit, unsigned).")
    lines.append("// Sampling rate (Hz): {}".format(int(fs)))
    lines.append("#pragma once")
    lines.append("#include <stdint.h>")
    lines.append("const unsigned int {}_FS = {};".format(var_name, int(fs)))
    lines.append("const unsigned int {}_N  = {};".format(var_name, data_uint8.size))
    lines.append("const uint8_t {}[{}] = {{".format(var_name, data_uint8.size))

    # Wrap 16 per line for readability
    for i in range(0, data_uint8.size, 16):
        chunk = ", ".join(str(v) for v in data_uint8[i:i+16])
        lines.append("  " + chunk + ("," if i + 16 < data_uint8.size else ""))

    lines.append("};")
    content = "\n".join(lines) + "\n"
    p = Path(header_path)
    p.write_text(content)
    return p

# --------------------------------------
# Example usage
# --------------------------------------
if __name__ == "__main__":
    out_wav = neurokit_ecg_to_wav8(
        filename="ecg_10s_8bit_500Hz.wav",
        fs=500,
        duration_s=10.0,
        heart_rate=70.0,
        method="ecgsyn"
    )
    print(f"WAV written to: {out_wav}")

    # If you also want a C header:
    # (Reload the bytes from the WAV or use the in-memory array.)
    # Here we re-create the same signal to export:
    import neurokit2 as nk
    fs = 500
    N = int(10 * fs)
    ecg = nk.ecg_simulate(length=N, sampling_rate=fs, method="ecgsyn", heart_rate=70.0)
    y_uint8 = to_uint8_pcm(ecg, peak=0.95)
    out_h = export_uint8_as_c_header(y_uint8, fs=fs, header_path="ecg_data.h", var_name="ecg_data")
    print(f"C header written to: {out_h}")


WAV written to: /home/stevend/Documents/SCRIPTS/LSTM/ecg_10s_8bit_500Hz.wav
C header written to: ecg_data.h
